In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
from keras.models import Sequential, Model
from keras.layers import Convolution2D, MaxPooling2D, Conv3D, MaxPooling3D, ZeroPadding3D
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization, Input
from keras.utils.np_utils import to_categorical
from sklearn.decomposition import PCA
from tensorflow.keras.optimizers import Nadam, SGD, Adadelta, RMSprop
from keras.optimizers import adam_v2
adam = adam_v2.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
import keras.callbacks as kcallbacks
from keras.regularizers import l2
import time
import collections
import pandas as pd
from sklearn import metrics, preprocessing
from Utils import zeroPadding, normalization, modelStatsRecord, averageAccuracy, cnn_3D,densenet_IN, densenet_IN_no_bottleneck_layer
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

# 产生新数据集的过程
# indexToAssignment(train_indices, whole_data.shape[0], whole_data.shape[1], PATCH_LENGTH)
# 训练集 ，151 ，151， 3
def indexToAssignment(index_, Row, Col, pad_length):
    new_assign = {}
    for counter, value in enumerate(index_):
        assign_0 = value // Col + pad_length
        assign_1 = value % Col + pad_length
        new_assign[counter] = [assign_0, assign_1]
    return new_assign

def assignmentToIndex(assign_0, assign_1, Row, Col):
    new_index = assign_0 * Col + assign_1
    return new_index

def selectNeighboringPatch(matrix, pos_row, pos_col, ex_len):
    selected_rows = matrix[range(pos_row - ex_len, pos_row + ex_len + 1), :]
    selected_patch = selected_rows[:, range(pos_col - ex_len, pos_col + ex_len + 1)]
    return selected_patch

# divide dataset into train and test datasets
def sampling(proptionVal, groundTruth):
    labels_loc = {}
    train = {}
    test = {}
    m = max(groundTruth)
    print(m)
    # 16
    for i in range(m):
        indices = [j for j, x in enumerate(groundTruth.ravel().tolist()) if x == i + 1]
        np.random.shuffle(indices)
        labels_loc[i] = indices
        nb_val = int(proptionVal * len(indices))
        train[i] = indices[:-nb_val]
        test[i] = indices[-nb_val:]
        
    train_indices = []
    test_indices = []
    for i in range(m):
        train_indices += train[i]
        test_indices += test[i]
    np.random.shuffle(train_indices)
    np.random.shuffle(test_indices)
    print(len(test_indices))
    # 8194
    print(len(train_indices))
    # 2055
    return train_indices, test_indices


def model_DenseNet():
    model_dense = cnn_3D.ResnetBuilder.build_resnet_8((1,img_rows, img_cols, img_channels), nb_classes)
    #model_dense = densenet_IN.ResnetBuilder.build_resnet_8((1, img_rows, img_cols, img_channels), nb_classes)

    RMS = RMSprop(lr=0.0001)#0.0003
    # Let's train the model using RMSprop
    model_dense.compile(loss='categorical_crossentropy', optimizer=RMS, metrics=['accuracy'])

    return model_dense


def dimension_PCA(data, input_dimension):
    dt = np.array(data,dtype='float32').reshape(-1,200)
    pca = PCA(n_components=input_dimension)
    dt = pca.fit_transform(dt)#data  
    whole_pca = np.zeros((data.shape[0], data.shape[1], input_dimension))


    for i in range(input_dimension):
         whole_pca[:, :, i] = dt[:,i].reshape(data.shape[0], data.shape[1])


    print (whole_pca.shape))

    return whole_pca


def dimension_AE(data,input_dimension):
    [d1,d2,d3]=data.shape
    data=data.reshape([d1*d2,d3])

    mean=data.mean()
    std=data.std()
    data=(data-mean)/std


    input_data = Input(shape=(200,))
    encoded = Dense(128,activation = 'relu')(input_data)
    encoded = Dense(input_dimension, activation='relu')(encoded)

    encoder = Model(input_data,encoded)
    encoder.summary()
    encoded_data = pd.DataFrame(encoder.predict(data))
    print(encoded_data.shape)
    
    whole_ae = np.zeros((d1, d2, input_dimension))

    for i in range(input_dimension):
         whole_ae[:, :, i] = encoded_data.values[:,i].reshape(d1, d2)

    print(whole_ae.shape)
    return whole_ae



mat_data = sio.loadmat('Houston.mat')
data_IN = mat_data['Houston']
#data_IN = dimension_PCA(data_IN,108)

mat_gt = sio.loadmat('Houston_gt.mat')
gt_IN = mat_gt['Houston_gt']
# print('data_IN:',data_IN)

#reduce dimension
#data_IN = dimension_PCA(data_IN,108)  #pca
#data_IN = dimension_AE(data_IN,108) #ae

print(data_IN.shape)
# (145,145,200)
print(gt_IN.shape)
# (145,145)

# new_gt_IN = set_zeros(gt_IN, [1,4,7,9,13,15,16])
new_gt_IN = gt_IN

batch_size = 8
nb_classes = 15
nb_epoch = 100 # 400
img_rows, img_cols = 7, 7  # 27, 27
patience = 200  #200

INPUT_DIMENSION_CONV = 144
INPUT_DIMENSION = 144



TOTAL_SIZE =15029 
VAL_SIZE = 1503

TRAIN_SIZE = 7518
TEST_SIZE = TOTAL_SIZE - TRAIN_SIZE
VALIDATION_SPLIT = 0.5  # 50% for trainnig and 50% for validation and testing


img_channels = 144
PATCH_LENGTH = 3  # Patch_size (13*2+1)*(13*2+1)

print(data_IN.shape[:2])

print(np.prod(data_IN.shape[:2]))

print(data_IN.shape[2:])

print(np.prod(data_IN.shape[2:]))

print(np.prod(new_gt_IN.shape[:2]))



data = data_IN.reshape(np.prod(data_IN.shape[:2]), np.prod(data_IN.shape[2:]))
gt = new_gt_IN.reshape(np.prod(new_gt_IN.shape[:2]), )


data = preprocessing.scale(data)
print(data.shape)
# (21025, 200)

# scaler = preprocessing.MaxAbsScaler()
# data = scaler.fit_transform(data)


data_ = data.reshape(data_IN.shape[0], data_IN.shape[1], data_IN.shape[2])
whole_data = data_
padded_data = zeroPadding.zeroPadding_3D(whole_data, PATCH_LENGTH)
print(padded_data.shape)
# (151, 151, 200)


ITER = 1
CATEGORY = 15

train_data = np.zeros((TRAIN_SIZE, 2 * PATCH_LENGTH + 1, 2 * PATCH_LENGTH + 1, INPUT_DIMENSION_CONV))
print(train_data.shape)
# (2055, 7, 7, 200)
test_data = np.zeros((TEST_SIZE, 2 * PATCH_LENGTH + 1, 2 * PATCH_LENGTH + 1, INPUT_DIMENSION_CONV))
print(test_data.shape)
# (8194, 7, 7, 200)


KAPPA_3DCNN = []
OA_3DCNN = []
AA_3DCNN = []
TRAINING_TIME_3DCNN = []
TESTING_TIME_3DCNN = []
ELEMENT_ACC_3DCNN = np.zeros((ITER, CATEGORY))

#seeds = [1220, 1221, 1222, 1223, 1224, 1225, 1226, 1227, 1228, 1229]

seeds = [1334]

for index_iter in range(ITER):
    print("# %d Iteration" % (index_iter + 1))
    # # 1 Iteration

    # save the best validated model
    best_weights_DenseNet_path = 'weights/IN/Houston_bestamdschaha_3DCNN514_' + str(
        index_iter + 1) + '.hdf5'

    # Get test and training samples through the sampling function.
    np.random.seed(seeds[index_iter])
    train_indices, test_indices = sampling(VALIDATION_SPLIT, gt)
    # train_indices 2055     test_indices 8094


    y_train = gt[train_indices] - 1
    y_train = to_categorical(np.asarray(y_train))

    y_test = gt[test_indices] - 1
    y_test = to_categorical(np.asarray(y_test))

 
    train_assign = indexToAssignment(train_indices, whole_data.shape[0], whole_data.shape[1], PATCH_LENGTH)
    for i in range(len(train_assign)):
        train_data[i] = selectNeighboringPatch(padded_data, train_assign[i][0], train_assign[i][1], PATCH_LENGTH)

    test_assign = indexToAssignment(test_indices, whole_data.shape[0], whole_data.shape[1], PATCH_LENGTH)
    for i in range(len(test_assign)):
        test_data[i] = selectNeighboringPatch(padded_data, test_assign[i][0], test_assign[i][1], PATCH_LENGTH)


    x_train = train_data.reshape(train_data.shape[0], train_data.shape[1], train_data.shape[2], INPUT_DIMENSION_CONV)
    x_test_all = test_data.reshape(test_data.shape[0], test_data.shape[1], test_data.shape[2], INPUT_DIMENSION_CONV)


    x_val = x_test_all[-VAL_SIZE:]
    y_val = y_test[-VAL_SIZE:]

    x_test = x_test_all[:-VAL_SIZE]
    y_test = y_test[:-VAL_SIZE]

    ############################################################################################################

    model_densenet = model_DenseNet()


    earlyStopping6 = kcallbacks.EarlyStopping(monitor='val_loss', patience=patience, verbose=1, mode='auto')
    # Users will save the model at the end of each epoch. If save_best_only=True, the last data with the latest verification error will be saved.
    saveBestModel6 = kcallbacks.ModelCheckpoint(best_weights_DenseNet_path, monitor='val_loss', verbose=1,
                                                save_best_only=True,
                                                mode='auto')

    # Training and verification
    tic6 = time.perf_counter()
    print(x_train.shape, x_test.shape)
    # (2055,7,7,200)  (7169,7,7,200)
    history_3d_densenet = model_densenet.fit(
        x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], x_train.shape[3], 1), y_train,
        validation_data=(x_val.reshape(x_val.shape[0], x_val.shape[1], x_val.shape[2], x_val.shape[3], 1), y_val),
        batch_size=batch_size,
        epochs=nb_epoch, shuffle=True, callbacks=[earlyStopping6, saveBestModel6])
    toc6 = time.perf_counter()
 
    # test
    tic7 = time.perf_counter()
    loss_and_metrics = model_densenet.evaluate(
        x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], x_test.shape[3], 1), y_test,
        batch_size=batch_size)
    toc7 = time.perf_counter()

    print('3DCNN Fit Time: ', toc6 - tic6)
    print('3DCNN Test time:', toc7 - tic7)

    print('3DCNN Test score:', loss_and_metrics[0])
    print('3DCNN Test accuracy:', loss_and_metrics[1])

    # print(history_3d_densenet.history.keys())
    # dict_keys(['val_loss', 'val_acc', 'loss', 'acc'])

    # predict
    pred_test = model_densenet.predict(
        x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], x_test.shape[3], 1)).argmax(axis=1)
    
    collections.Counter(pred_test)

    gt_test = gt[test_indices] - 1
    # print(len(gt_test))
    # 8194
    overall_acc = metrics.accuracy_score(pred_test, gt_test[:-VAL_SIZE])
    confusion_matrix = metrics.confusion_matrix(pred_test, gt_test[:-VAL_SIZE])
    print(confusion_matrix)
    print("3DCNN finished.")
    print("# %d Iteration" % (index_iter + 1))


